In [ ]:
options(jupyter.rich_display = F)

# LAB 8: MATRICES

**by Serhat Çevikel** 

## NEIGHBOUR CITIES

We have a 81x81 matrix of bird fly distances in km's between 81 province centers in Turkey. To retrieve this matrix please follw the link below to download the file distance.RData:

[distance.RData](../file/distance.RData)

After you download the file to your local computer, load the data as such:

In [ ]:
load("~/file/distance.RData")

In [ ]:
distance[1:10, 1:10]

In [ ]:
distance[1:20, c("İstanbul", "Ankara", "İzmir", "Adana")]

Now write a function named **neighbour** that takes three arguments:

- city: Either a city code as integer (i.e. 34) or city name (i.e. "İstanbul")
- radius: Radius in km within which we want to filter the neighbours of the city
- dist: A distance matrix, default value of which is the "distance" global object

as such:

```R
neighbour(34, 150)

  Yalova  Kocaeli    Bursa Tekirdağ  Sakarya  Bilecik 
      47       78       91      123      127      128
```

```R
neighbour(city = "Adana", radius = 100)

  Mersin Osmaniye 
      64       82

neighbour(city = "İzmir", 50)

  Manisa 
      33
```

Hints:
- Inside the function, first filter the row or column of the matrix for the city. We then have a vector
- Then filter the vector such that only those values below or equal to the radius **AND** above 0 are left. Why do we filter out 0 km?
- Then sort and return the remaining neighbours

### SOLUTION

In [ ]:
neighbour <- function(city, radius, dist = distance)
{
    row <- dist[city,]
    neighs <- row[row <= radius & row > 0]
    return(sort(neighs))
}

neighbour(34, 150)

neighbour(city = "Adana", radius = 100)

neighbour(city = "İzmir", 50)

## NEIGHBOUR COUNTS

Now, use the same distance object.

Write a function named **neigh_count** that takes two arguments:

- radius
- dist with a default value of distance object

The function should return the count of neighbouring cities of each city within a radius, sorted descending

Hint: Use apply function

### SOLUTION

In [ ]:
neigh_count <- function(radius, dist = distance)
{
    counts <- apply(dist, 1, function(x) sum(x <= radius) - 1)
    return(sort(counts, decreasing = T))
}
                    
neigh_count(100)

## SQUARE DONUT

You are supposed to write a function named "square_donut" that takes two arguments n and x:
- That creates a square matrix of size n
- The outer x squares of the matrix should have 1, while the square "hole" in the middle should be all zeros
- If x is greater than or equal to the half of n, it should throw a message as "edge too wide" and stop

So:
```R
square_donut(7, 2)
```
returns
```
1	1	1	1	1	1	1
1	1	1	1	1	1	1
1	1	0	0	0	1	1
1	1	0	0	0	1	1
1	1	0	0	0	1	1
1	1	1	1	1	1	1
1	1	1	1	1	1	1
```

```R
square_donut(6, 3)
```
returns
```
'edge too wide'
```

### SOLUTION

In [ ]:
square_donut <- function(n, x)
{
    if (x >= n / 2)
    {
        return("edge too wide")
    }
    
    mat1 <- matrix(1, nrow = n, ncol = n) # create matrix of 1's
    startx <- x + 1 # start index of 0's
    endx <- n - x # end index of 0's
    mat1[startx:endx, startx:endx] <- 0 # create the hole of 0's
    
    return(mat1)
}

square_donut(7, 2)
square_donut(6, 3)

## MINEFIELD

The below code creates a sample matrix that represents a minefield: 0 means an empty cell, 1 means a cell with a mine

- size: The size of the matrix (row or column count)
- density: The ratio of 1 cells to all cells

In [ ]:
new_field <- function(size = 5, density = 0.2)
{
    vec <- sample(1:0,
                  size^2,
                  replace = T,
                  prob = c(density, 1 - density))
    mat <- matrix(vec, nrow = size)
    return(mat)
}

set.seed(1)
field1 <- new_field()
field1

Now write a function named minefield that takes 3 arguments:

- rw: An integer for row index
- cl: An integer for column index
- field: A matrix for the minefield. Default value is the field1 global object

The function should first retrieve the value of the cell at the rw and cl indices.

If the value is equal to 1, there is a hit, and the function should return 9 (that represents a hit)
Else, there is a miss, then we should get the number of neighbouring hits: Meaning the count of 1's in neighbouring eight cells of the target

To do that, we should get the the neighbouring row indices. If the target row index is 2, then the neighbouring row indices are 1:3. Do the same for column indices

The function should then subset the field matrix for the neighbouring row and column indices, and get the sum of the values in this subset.


In this first version the row or column indices should not be 1 or the size of the matrix otherwise the function will return a subscript out of bound error.

In a more elegant version, the neighbouring row and column indices can be filtered for the values above 0 and below or equal to the size of the matrix

as such:

```R
minefield(1,1)
[1] 2

minefield(2,2)
[1] 9

minefield(2,3)
[1] 3
```

### SOLUTION

In [ ]:
minefield <- function(rw, cl, field = field1)
{
    val <- field[rw, cl] # get the value
        
    if (val == 1) # there is a hit
    {
        return(9)
    }
    else # there is a miss
    {
        size <- nrow(field) # max index allowed

        rws <- (-1):1 + rw # neighbouring row indices
        rws <- rws[rws > 0 & rws <= size] # filter between 1 and size
        
        cls <- (-1):1 + cl  # neighbouring column indices
        cls <- cls[cls > 0 & cls <= size] # filter between 1 and size
        
        sub <- field[rws, cls] # subset the neighours
        return(sum(sub)) # return the sum of neighbouring hits
    }
}

minefield(1,1)
minefield(2,2)
minefield(2,3)

## RIDDLE DOWN THE MATRIX

You are given a matrix generator function as such:

```R
matrix_gen <- function(maxx = 100, nr = 5, nc = 5)
{
    matrix(sample(maxx, nr * nc, replace = T), nrow = nr) 
}
```

```R
Using this matrix, you will write a function as such:

max_path <- function(matt)
{

...

}
```


That will return the maximum sum to be attained when a path is selected from the first row to the end, in each step of which you can select any of the adjacent cells (straight down, down left, down right) on the next row.

Let's say, for a random 3x3 matrix of 1:10 values:

```R
mattt <- matrix_gen(10, 3, 3)
mattt
     [,1] [,2] [,3]
[1,]    9    3    4
[2,]    4    4    3
[3,]    6    5    9
```
```R
max_path(mattt)
[1] 22
```

(I guess when you follow the primary diagon)


Or when:

```R
mattt <- matrix_gen(10, 5, 5)
mattt
     [,1] [,2] [,3] [,4] [,5]
[1,]    2    8    2   10    9
[2,]    3    5    5   10    3
[3,]    9    8   10   10   10
[4,]    8    5    6    7    5
[5,]    5    1    6    9    1
```

The result is:

```
max_path(mattt)
[1] 46
```

(I guess it is the 4th column)

HINT: You may make use of pmax() function or just apply()

### SOLUTION

In [ ]:
matrix_gen <- function(maxx = 100, nr = 5, nc = 5)
{
    matrix(sample(maxx, nr * nc, replace = T), nrow = nr)
}

max_path <- function(matt)
{
    # we will track a vector length of which is equal to the number of the rows and
    # that shows the max sums for each cell in the current row up to that row
    # we take the first row as the initial value for this vector (the values show the maximum sum for each column up to that point!)
    sums_vec <- matt[1,]

    # get the dimensions of the original matrix. can also be done with dim()
    nc <- ncol(matt)
    nr <- nrow(matt)
    
    # across rows, starting from the second one
    # on the first row the max values up to that row are the values themselves
    for (roww in 2:nr) 
    {
        # takes three vector:
        # the original sums_vec, the sums_vec offset to left and offset to right
        # this is done by adding a zero to left/trimming from right 
        # or adding a zero to right/trimming from left
        # why? we can come to a cell from straight up, left up or right up
        # so for each cell in the row we should the select the max of these three values
        # pmax does it pairwise (or triple wise in this case)
        # this is the critical part of the code
        max_of_sums_vec <- pmax(c(0, sums_vec[-nc]), sums_vec, c(sums_vec[-1], 0))

        # update the sums vec: add the maximum sums vector (that shows the maximum sums up to that row for each cell)
        # to the current row
        sums_vec <- matt[roww,] + max_of_sums_vec
    }

    # we have come to the last row
    # now just take the maximum of the last sums_vec
    maxval <- max(sums_vec)

    return(maxval)

}

mat1 <- matrix_gen(10, 3, 3)
mat1
max_path(mat1)